<a href="https://colab.research.google.com/github/auliaslsblc/nlpcc-ui-2025/blob/main/Class_4_Assignment_Synthetic_Data%2C_Embeddings%2C_and_Semantic_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis and Semantic Search on Coffeeshop Reviews Using OpenAI Embeddings**

This project focuses on analyzing cafe customer reviews, starting by structuring a dataset of reviews with their corresponding topics and sentiment labels. Subsequently, each review is transformed into a numerical embedding vector using an OpenAI model to capture its semantic meaning. Finally, these embeddings are leveraged to build an intelligent search system capable of finding the most relevant reviews based on user queries, utilizing cosine similarity as the measure of semantic likeness.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd

# --- GANTI DENGAN PATH YANG SUDAH DIVERIFIKASI ---
# Berdasarkan output os.listdir(), buat path yang benar di sini:
# Contoh jika file ada di MyDrive:
# path_ke_file = '/content/drive/MyDrive/drive/Topic_Label_Coffeeshop.csv'
# Contoh jika file ada di subfolder 'FolderSaya' di MyDrive:
# path_ke_file = '/content/drive/MyDrive/drive/Topic_Label_Coffeeshop.csv'
path_ke_file = '/content/drive/MyDrive/drive/Topic_Label_Coffeeshop.csv'
try:
    df_ulasan = pd.read_csv(path_ke_file)
    print(f"DataFrame berhasil dimuat dari: {path_ke_file}")
    print("Jumlah baris dan kolom:", df_ulasan.shape)
    print(df_ulasan.head())
    # ... sisa kode Anda ...

except FileNotFoundError:
    print(f"ERROR: File MASIH tidak ditemukan di path '{path_ke_file}'.")
    print("Mohon periksa kembali output dari os.listdir() dan pastikan path dan nama file 100% akurat, termasuk huruf besar/kecil dan spasi.")
except Exception as e:
    print(f"Terjadi kesalahan saat membaca CSV: {e}")

DataFrame berhasil dimuat dari: /content/drive/MyDrive/drive/Topic_Label_Coffeeshop.csv
Jumlah baris dan kolom: (200, 3)
                                         Teks Ulasan              Topik  \
0  Tempatnya bener-bener bikin betah! Dikelilingi...  Suasana dan Vibes   
1  Vibesnya asri dan sejukcocok buat healing atau...  Suasana dan Vibes   
2  Suka banget sama konsepnya yang hijauudaranya ...  Suasana dan Vibes   
3  Suasana di sini tuh tenang bangetbikin pikiran...  Suasana dan Vibes   
4  Kalau cari tempat ngopi yang adem dan rindangd...  Suasana dan Vibes   

     Label  
0  Positif  
1  Positif  
2  Positif  
3  Positif  
4  Positif  


# Task 2: Text Embedding Generation
The goal of this step is to convert each "Review Text" from your synthetic dataset into a numerical embedding vector using the specified model.

In [ ]:
# Sel 1: Mengambil API Key dari Colab Secrets
from google.colab import userdata
import openai # Import openai di sini agar bisa langsung digunakan setelah key didapat

try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    if not openai_api_key:
        raise ValueError("API key 'OPENAI_API_KEY' tidak ditemukan di Colab Secrets.")
    print("OpenAI API Key berhasil diambil dari Colab Secrets.")

    # Inisialisasi OpenAI client di sini setelah API key berhasil didapatkan
    client = openai.OpenAI(api_key=openai_api_key)
    print("OpenAI client berhasil diinisialisasi.")

except Exception as e:
    print(f"Error: {e}")
    print("Pastikan Anda sudah menambahkan API key Anda ke Colab Secrets dengan nama 'OPENAI_API_KEY' dan mengaktifkan 'Notebook access'.")
    client = None # Set client ke None jika gagal

OpenAI API Key berhasil diambil dari Colab Secrets.
OpenAI client berhasil diinisialisasi.


In [ ]:
# Sel 2: Instal Pustaka
!pip install openai pandas tqdm

In [ ]:
# Sel 3: Impor Pustaka Tambahan
import pandas as pd
from tqdm import tqdm
import os # Meskipun key sudah di-handle, os bisa berguna untuk hal lain

In [ ]:
# Sel 4 (Opsi A): Unggah File CSV
from google.colab import files

print("Silakan unggah file CSV Anda yang berisi 'Teks Ulasan'.")
uploaded = files.upload()

if uploaded:
    file_name = next(iter(uploaded)) # Mengambil nama file pertama yang diunggah
    print(f"\nFile '{file_name}' berhasil diunggah.")
    try:
        df = pd.read_csv(file_name)
        print("DataFrame berhasil dimuat.")
        print("Contoh data:")
        print(df.head())
    except Exception as e:
        print(f"Error saat memuat CSV ke DataFrame: {e}")
        df = None # Set df ke None jika gagal
else:
    print("Tidak ada file yang diunggah.")
    df = None

Silakan unggah file CSV Anda yang berisi 'Teks Ulasan'.


Saving Topic_Label_Coffeeshop.csv to Topic_Label_Coffeeshop.csv

File 'Topic_Label_Coffeeshop.csv' berhasil diunggah.
DataFrame berhasil dimuat.
Contoh data:
                                         Teks Ulasan              Topik  \
0  Tempatnya bener-bener bikin betah! Dikelilingi...  Suasana dan Vibes   
1  Vibesnya asri dan sejukcocok buat healing atau...  Suasana dan Vibes   
2  Suka banget sama konsepnya yang hijauudaranya ...  Suasana dan Vibes   
3  Suasana di sini tuh tenang bangetbikin pikiran...  Suasana dan Vibes   
4  Kalau cari tempat ngopi yang adem dan rindangd...  Suasana dan Vibes   

     Label  
0  Positif  
1  Positif  
2  Positif  
3  Positif  
4  Positif  


In [ ]:
# Sel 5: Fungsi untuk Menghasilkan Embedding
# Pilih model embedding OpenAI Anda (sesuaikan dengan instruksi kelas atau preferensi)
MODEL_EMBEDDING = "text-embedding-3-small"  # Model efisien dan cukup baik
# MODEL_EMBEDDING = "text-embedding-ada-002" # Model generasi sebelumnya yang juga populer

def get_embeddings_openai_batch(texts_list, model=MODEL_EMBEDDING, batch_size=200):
    """
    Menghasilkan embedding untuk daftar teks dalam batch menggunakan OpenAI API.
    texts_list: list dari string teks.
    model: nama model embedding OpenAI.
    batch_size: jumlah teks yang dikirim per panggilan API.
    """
    if client is None:
        print("OpenAI client tidak diinisialisasi. Tidak dapat menghasilkan embedding.")
        return None
    if not texts_list:
        print("Daftar teks kosong, tidak ada embedding yang dihasilkan.")
        return []

    all_embeddings = []
    print(f"Memulai pembuatan embedding untuk {len(texts_list)} teks dengan model '{model}' dan batch size {batch_size}...")

    for i in tqdm(range(0, len(texts_list), batch_size), desc="Memproses Batch Embedding"):
        batch_texts = texts_list[i:i + batch_size]
        # Pastikan tidak ada teks None atau kosong murni, ganti dengan spasi jika ada
        processed_batch = [str(text).strip() if text and str(text).strip() else " " for text in batch_texts]

        try:
            response = client.embeddings.create(
                input=processed_batch,
                model=model
            )
            embeddings_for_batch = [item.embedding for item in response.data]
            all_embeddings.extend(embeddings_for_batch)
        except Exception as e:
            print(f"Error pada batch {i//batch_size + 1}: {e}")
            # Tambahkan None untuk setiap item dalam batch yang gagal agar panjangnya tetap sesuai
            all_embeddings.extend([None] * len(batch_texts))

    print(f"Selesai membuat embedding. Total embedding dihasilkan/dicoba: {len(all_embeddings)}")
    return all_embeddings

In [ ]:
# Sel 6: Hasilkan Embedding dan Simpan
if df is not None and 'Teks Ulasan' in df.columns:
    print(f"\nMemulai proses embedding untuk kolom 'Teks Ulasan' ({len(df)} baris)...")

    # Pastikan kolom 'Teks Ulasan' adalah string dan tangani nilai NaN
    df['Teks Ulasan'] = df['Teks Ulasan'].astype(str).fillna(" ")
    list_of_reviews = df['Teks Ulasan'].tolist()

    # Hasilkan embedding
    embeddings_hasil = get_embeddings_openai_batch(list_of_reviews)

    if embeddings_hasil and len(embeddings_hasil) == len(df):
        df['embedding_openai'] = embeddings_hasil
        print("\nKolom 'embedding_openai' berhasil ditambahkan ke DataFrame.")

        # Cek berapa banyak embedding yang berhasil (bukan None)
        valid_embeddings_count = df['embedding_openai'].count() # count() pada series pandas mengabaikan None
        print(f"Jumlah embedding yang valid (non-None): {valid_embeddings_count} dari {len(df)} ulasan.")
        if valid_embeddings_count < len(df):
            print("Beberapa embedding mungkin gagal dibuat (bernilai None). Periksa output error di atas.")

        print("\nContoh data dengan embedding:")
        # Tampilkan hanya kolom relevan dan beberapa baris agar tidak terlalu panjang
        print(df[['Teks Ulasan', 'Label', 'embedding_openai']].head())

        # Simpan DataFrame yang sudah ada embeddingnya
        # Format pickle lebih baik untuk menyimpan list/array dalam sel DataFrame
        output_filename_pkl = 'dataset_dengan_embedding_openai.pkl'
        try:
            df.to_pickle(output_filename_pkl)
            print(f"\nDataFrame dengan embedding berhasil disimpan sebagai '{output_filename_pkl}'.")
            # Anda bisa mengunduhnya dari Colab jika perlu:
            # files.download(output_filename_pkl)
        except Exception as e:
            print(f"Error saat menyimpan DataFrame ke pickle: {e}")

    elif embeddings_hasil is None:
        print("\nProses embedding tidak menghasilkan apa-apa, kemungkinan karena client OpenAI tidak siap.")
    else:
        print("\nJumlah embedding yang dihasilkan tidak sesuai dengan jumlah baris DataFrame. Silakan periksa error.")

elif df is None:
    print("\nDataFrame 'df' belum dimuat. Silakan jalankan sel untuk memuat data terlebih dahulu.")
else:
    print("\nKolom 'Teks Ulasan' tidak ditemukan dalam DataFrame. Pastikan nama kolom sudah benar.")


Memulai proses embedding untuk kolom 'Teks Ulasan' (200 baris)...
Memulai pembuatan embedding untuk 200 teks dengan model 'text-embedding-3-small' dan batch size 200...


Memproses Batch Embedding: 100%|██████████| 1/1 [00:03<00:00,  3.02s/it]

Selesai membuat embedding. Total embedding dihasilkan/dicoba: 200

Kolom 'embedding_openai' berhasil ditambahkan ke DataFrame.
Jumlah embedding yang valid (non-None): 200 dari 200 ulasan.

Contoh data dengan embedding:
                                         Teks Ulasan    Label  \
0  Tempatnya bener-bener bikin betah! Dikelilingi...  Positif   
1  Vibesnya asri dan sejukcocok buat healing atau...  Positif   
2  Suka banget sama konsepnya yang hijauudaranya ...  Positif   
3  Suasana di sini tuh tenang bangetbikin pikiran...  Positif   
4  Kalau cari tempat ngopi yang adem dan rindangd...  Positif   

                                    embedding_openai  
0  [0.011053933762013912, -0.00456699263304472, -...  
1  [-0.014044753275811672, 0.005080667324364185, ...  
2  [-0.018063683062791824, -0.023671826347708702,...  
3  [0.05218210816383362, -0.011847035959362984, -...  
4  [-0.008472133427858353, -0.005187409929931164,...  

DataFrame dengan embedding berhasil disimpan sebagai 'datas

# **TASK 3: EMBEDDING ANALYSIS**
Semantic search goal: build a system that can accept a text query from a user, then find and rank the reviews from your dataset that are most similar in meaning (semantics) to the query. Similarity is measured using cosine similarity between the query embedding and the embedding of each review in the dataset.

In [ ]:
# Sel 7: Instal Pustaka (jika ada yang belum)
!pip install scikit-learn numpy

In [ ]:
# Sel 8: Impor Pustaka
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Pastikan client OpenAI dan MODEL_EMBEDDING dari sel sebelumnya sudah ada
# import openai # jika belum diimpor di sesi ini
# MODEL_EMBEDDING = "text-embedding-3-small" # atau model yang Anda gunakan

In [ ]:
# Sel 9: Persiapkan Embedding dari DataFrame
if 'df' not in locals() or df is None or 'embedding_openai' not in df.columns:
    print("DataFrame 'df' atau kolom 'embedding_openai' tidak ditemukan. Mohon jalankan Tugas 2 terlebih dahulu.")
else:
    # Hapus baris di mana embedding mungkin None (jika ada kegagalan parsial di Tugas 2)
    df_filtered = df.dropna(subset=['embedding_openai']).copy() # Buat salinan untuk menghindari SettingWithCopyWarning

    if df_filtered.empty:
        print("Tidak ada data dengan embedding yang valid untuk diproses.")
    else:
        # Konversi list embedding menjadi matriks NumPy
        # Pastikan setiap elemen adalah list/array angka dan bukan string atau None
        valid_embeddings_list = [emb for emb in df_filtered['embedding_openai'].tolist() if isinstance(emb, (list, np.ndarray))]

        if not valid_embeddings_list:
            print("Tidak ditemukan embedding yang valid dalam format list atau array NumPy.")
        else:
            # Cek apakah semua embedding memiliki dimensi yang sama
            first_embedding_dim = len(valid_embeddings_list[0])
            if not all(len(emb) == first_embedding_dim for emb in valid_embeddings_list):
                print("Dimensi embedding tidak konsisten. Pastikan semua embedding memiliki panjang yang sama.")
                # Anda mungkin perlu memfilter lebih lanjut atau memperbaiki data embedding Anda.
            else:
                document_embeddings = np.array(valid_embeddings_list)
                print(f"Matriks embedding dokumen berhasil dibuat dengan shape: {document_embeddings.shape}")
                # Simpan juga teks yang sesuai dengan embedding ini untuk ditampilkan
                corresponding_texts = df_filtered['Teks Ulasan'].tolist()
                corresponding_labels = df_filtered['Label'].tolist() # Jika ingin menampilkan label juga

Matriks embedding dokumen berhasil dibuat dengan shape: (200, 1536)


In [ ]:
# Sel 10: Fungsi untuk Mendapatkan Embedding Kueri
def get_query_embedding(query_text, model=MODEL_EMBEDDING):
    """
    Menghasilkan embedding untuk satu teks kueri.
    """
    if client is None:
        print("Client OpenAI tidak diinisialisasi.")
        return None
    try:
        # Ganti string kosong dengan spasi untuk menghindari error API
        processed_query = query_text if query_text and query_text.strip() else " "
        response = client.embeddings.create(
            input=[processed_query], # Kirim sebagai list walau hanya satu item
            model=model
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error saat membuat embedding untuk kueri '{query_text}': {e}")
        return None

In [ ]:
# Sel 11: Fungsi Pencarian Semantik
def semantic_search(query_text, top_n=5):
    """
    Melakukan pencarian semantik dan mengembalikan top_n hasil yang paling mirip.
    """
    if 'document_embeddings' not in globals() or document_embeddings is None or document_embeddings.size == 0:
        print("Embedding dokumen tidak tersedia. Jalankan persiapan embedding terlebih dahulu.")
        return

    query_embedding = get_query_embedding(query_text)
    if query_embedding is None:
        return

    # Reshape query_embedding menjadi 2D array untuk cosine_similarity
    query_embedding_2d = np.array(query_embedding).reshape(1, -1)

    # Hitung cosine similarity
    similarities = cosine_similarity(query_embedding_2d, document_embeddings)

    # Dapatkan skor similarity untuk setiap dokumen (flatten array)
    similarity_scores = similarities[0]

    # Urutkan dokumen berdasarkan similarity (dari tertinggi ke terendah)
    # argsort mengembalikan indeks, [::-1] membalik urutannya
    sorted_indices = np.argsort(similarity_scores)[::-1]

    print(f"\nHasil Pencarian Cerdas untuk kueri: '{query_text}' (Top {top_n}):")
    print("-------------------------------------------------------------")
    for i in range(min(top_n, len(sorted_indices))):
        idx = sorted_indices[i]
        print(f"Peringkat {i+1}:")
        print(f"  Teks   : {corresponding_texts[idx][:200]}...") # Tampilkan 200 karakter pertama
        print(f"  Label  : {corresponding_labels[idx]}") # Tampilkan label sintetis
        print(f"  Skor Similarity: {similarity_scores[idx]:.4f}")
        print("---")

In [ ]:
# Sel 12: Demonstrasi Pencarian
# Pastikan sel-sel sebelumnya (terutama Sel 9, 10, 11) sudah dijalankan dan 'document_embeddings' serta 'client' sudah siap.

# Contoh Kueri:
query1 = "kopi yang rasanya kuat dan pahit"
semantic_search(query1, top_n=3)

query2 = "tempat ngopi yang tenang untuk kerja"
semantic_search(query2, top_n=3)

query3 = "pelayanan lama dan tidak ramah"
semantic_search(query3, top_n=3)

query4 = "harga mahal tidak sepadan"
semantic_search(query4, top_n=3)

query5 = "wifi lemot bikin kesal"
semantic_search(query5, top_n=3)


Hasil Pencarian Cerdas untuk kueri: 'kopi yang rasanya kuat dan pahit' (Top 3):
-------------------------------------------------------------
Peringkat 1:
  Teks   : Kopi hitamnya terlalu pahit gosongnggak enjoyable....
  Label  : Negatif
  Skor Similarity: 0.6915
---
Peringkat 2:
  Teks   : Harganya sebanding sama rasa kopi dan suasana yang didapat....
  Label  : Positif
  Skor Similarity: 0.6258
---
Peringkat 3:
  Teks   : Rasa kopinya otentik dan berkesan....
  Label  : Positif
  Skor Similarity: 0.6148
---

Hasil Pencarian Cerdas untuk kueri: 'tempat ngopi yang tenang untuk kerja' (Top 3):
-------------------------------------------------------------
Peringkat 1:
  Teks   : Kalau cari tempat ngopi yang adem dan rindangdi sini juaranya....
  Label  : Positif
  Skor Similarity: 0.6468
---
Peringkat 2:
  Teks   : Nggak nyangka ada tempat sehijau dan setenang ini buat ngopi....
  Label  : Positif
  Skor Similarity: 0.6030
---
Peringkat 3:
  Teks   : WiFi kencangcocok buat kerja atau n

# **Task 4: API Key Management**
An important aspect of working with cloud APIs is securely managing your API keys.

In [ ]:
# Sel untuk Pengaturan Aman API Key OpenAI
from google.colab import userdata
import os # Opsional, jika Anda ingin menyimpannya juga sebagai environment variable

# Nama secret yang Anda gunakan di Colab Secrets
NAMA_SECRET_OPENAI_KEY = 'OPENAI_API_KEY' # Ini adalah nama yang Anda konfirmasi

try:
    # Mengambil API Key OpenAI dari Colab Secrets
    openai_api_key = userdata.get(NAMA_SECRET_OPENAI_KEY)

    if openai_api_key:
        print(f"API Key '{NAMA_SECRET_OPENAI_KEY}' berhasil dimuat dari Colab Secrets.")
        # Anda bisa menggunakannya untuk konfigurasi API OpenAI, contoh:
        # import openai
        # openai.api_key = openai_api_key
        # print("Konfigurasi API OpenAI dengan kunci yang dimuat berhasil.")

        # (Opsional) Menyimpan sebagai environment variable untuk sesi ini
        # os.environ['OPENAI_API_KEY'] = openai_api_key
        # print(f"API Key juga telah diatur sebagai environment variable 'OPENAI_API_KEY' untuk sesi ini.")

    else:
        print(f"❌ Gagal memuat API Key '{NAMA_SECRET_OPENAI_KEY}'.")
        print("Pastikan Anda telah menambahkan secret dengan nama yang benar ('OPENAI_API_KEY') dan mengaktifkan 'Notebook access' di panel Secrets.")

except Exception as e:
    # Perhatikan baris di bawah ini, pastikan stringnya lengkap
    print(f"Terjadi kesalahan saat mencoba mengakses API Key '{NAMA_SECRET_OPENAI_KEY}': {e}")
    print("Pastikan panel 'Secrets' sudah dikonfigurasi dengan benar.") # <-- PERBAIKAN DI SINI

# PENTING: JANGAN PERNAH MENULISKAN API KEY ANDA SECARA LANGSUNG DI SINI
# contoh_api_key_SALAH = "sk-***********************************" # <= JANGAN LAKUKAN INI!

API Key 'OPENAI_API_KEY' berhasil dimuat dari Colab Secrets.


Secure API Key Handling Storing API keys securely is vital to prevent unauthorized access and misuse.

Colab Secrets (Secure): This method is secure because your API key is not written directly in your notebook's code. It's stored separately by Colab and accessed via userdata.get(). This prevents accidental exposure if you share your notebook or commit it to version control, as the secret value itself remains hidden.

Hardcoding API Keys (Bad Practice): Hardcoding (e.g., api_key = "sk-..." in your code) is bad because it directly embeds your secret key into the code. This makes it highly vulnerable to exposure if the code is shared, committed to Git/GitHub, or even just displayed. Once exposed, the key can be easily stolen and misused.